In [34]:
import pandas as pd, numpy as np, dateutil.easter as easter
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator, FormatStrFormatter, PercentFormatter
from sklearn.linear_model import LinearRegression
from sklearn.compose import TransformedTargetRegressor

In [35]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.head(1)

,row_id,date,country,store,product,num_sold
0,0,2015-01-01,Finland,KaggleMart,Kaggle Mug,329


In [36]:
test_df.head(1)

,row_id,date,country,store,product
0,26298,2019-01-01,Finland,KaggleMart,Kaggle Mug


In [37]:
# convert date column to datetime in both train and test dataset
for df in [train_df, test_df]:
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True, drop=False)

In [38]:
train_df.head(1)

,row_id,date,country,store,product,num_sold
date,,,,,,
2015-01-01,0,2015-01-01,Finland,KaggleMart,Kaggle Mug,329


In [39]:
# get count of unique countries, stores, products
print(f"Number of countries in training dataset: {len(train_df['country'].unique())}")
print(f"Number of stores in training dataset: {len(train_df['store'].unique())}")
print(f"Number of products in training dataset: {len(train_df['product'].unique())}")
print(f"Total number of possible combinations: {len(train_df['country'].unique()) * len(train_df['store'].unique()) * len(train_df['product'].unique())}")
print(f"Total number of unique days in training dataset: {len(train_df.index.unique())}")


Number of countries in training dataset: 3
Number of stores in training dataset: 2
Number of products in training dataset: 3
Total number of possible combinations: 18
Total number of unique days in training dataset: 1461


In [40]:
print(f"All available years in training dataset: {list((train_df.index.year).unique())}")

All available years in training dataset: [2015, 2016, 2017, 2018]


In [41]:
train_df.groupby(['country', 'store', 'product']).agg({'date':'count'}).rename(columns = {'date':'date_count'})

date_count
country store      product                   
Finland KaggleMart Kaggle Hat            1461
                   Kaggle Mug            1461
                   Kaggle Sticker        1461
        KaggleRama Kaggle Hat            1461
                   Kaggle Mug            1461
                   Kaggle Sticker        1461
Norway  KaggleMart Kaggle Hat            1461
                   Kaggle Mug            1461
                   Kaggle Sticker        1461
        KaggleRama Kaggle Hat            1461
                   Kaggle Mug            1461
                   Kaggle Sticker        1461
Sweden  KaggleMart Kaggle Hat            1461
                   Kaggle Mug            1461
                   Kaggle Sticker        1461
        KaggleRama Kaggle Hat            1461
                   Kaggle Mug            1461
                   Kaggle Sticker        1461

In [42]:
print(f"First day: {train_df.date.min()} \nLast day: {train_df.date.max()}")

First day: 2015-01-01 00:00:00 
Last day: 2018-12-31 00:00:00


In [43]:
train_df.groupby(['country', 'store', 'product']).num_sold.agg(['min', 'max', 'mean'])

min   max         mean
country store      product                               
Finland KaggleMart Kaggle Hat      210  1113   362.479808
                   Kaggle Mug      126   774   204.200548
                   Kaggle Sticker   70   326   103.044490
        KaggleRama Kaggle Hat      354  1895   628.926762
                   Kaggle Mug      220  1398   356.110883
                   Kaggle Sticker  128   559   180.232033
Norway  KaggleMart Kaggle Hat      335  1809   594.645448
                   Kaggle Mug      201  1113   334.370294
                   Kaggle Sticker  114   518   169.577687
        KaggleRama Kaggle Hat      596  2884  1036.357974
                   Kaggle Mug      366  1935   584.297741
                   Kaggle Sticker  214   874   295.607803
Sweden  KaggleMart Kaggle Hat      248  1207   419.214237
                   Kaggle Mug      149   730   235.885010
                   Kaggle Sticker   86   356   119.613279
        KaggleRama Kaggle Hat      428  2169   731.452430
                   Kaggle Mug      253  1438   411.273101
                   Kaggle Sticker  148   637   208.314853

In [44]:
test_df.date.min(), test_df.date.max()

(Timestamp('2019-01-01 00:00:00'), Timestamp('2019-12-31 00:00:00'))

In [46]:
kk = train_df.groupby(['country', 'store', 'product']).num_sold.mean().unstack(level='store')
kk['KaggleRama / KaggleMart'] = round(kk.KaggleRama / kk.KaggleMart,2)
kk

store                   KaggleMart   KaggleRama  KaggleRama / KaggleMart
country product                                                         
Finland Kaggle Hat      362.479808   628.926762                     1.74
        Kaggle Mug      204.200548   356.110883                     1.74
        Kaggle Sticker  103.044490   180.232033                     1.75
Norway  Kaggle Hat      594.645448  1036.357974                     1.74
        Kaggle Mug      334.370294   584.297741                     1.75
        Kaggle Sticker  169.577687   295.607803                     1.74
Sweden  Kaggle Hat      419.214237   731.452430                     1.74
        Kaggle Mug      235.885010   411.273101                     1.74
        Kaggle Sticker  119.613279   208.314853                     1.74

In [47]:
# Group by year
kk = train_df.groupby(['country', 'store', 'product', train_df.date.dt.year]).num_sold.mean().unstack(level='product')
kk['Mugs/Sticker'] = kk['Kaggle Mug'] / kk['Kaggle Sticker']
kk['Hats/Sticker'] = kk['Kaggle Hat'] / kk['Kaggle Sticker']
kk

product                   Kaggle Hat  Kaggle Mug  Kaggle Sticker  \
country store      date                                            
Finland KaggleMart 2015   328.947945  185.657534       93.517808   
                   2016   342.773224  192.155738       97.174863   
                   2017   372.758904  210.479452      106.082192   
                   2018   405.493151  228.542466      115.419178   
        KaggleRama 2015   573.260274  322.868493      163.813699   
                   2016   596.931694  335.158470      169.871585   
                   2017   644.789041  367.315068      185.936986   
                   2018   700.813699  399.158904      201.334247   
Norway  KaggleMart 2015   570.076712  320.043836      161.936986   
                   2016   545.489071  306.849727      156.221311   
                   2017   601.043836  337.942466      171.227397   
                   2018   662.106849  372.720548      188.961644   
        KaggleRama 2015   991.465753  556.156164      281.635616   
                   2016   952.767760  538.267760      272.286885   
                   2017  1046.578082  592.904110      299.679452   
                   2018  1154.849315  649.989041      328.893151   
Sweden  KaggleMart 2015   396.342466  222.854795      112.863014   
                   2016   407.322404  227.469945      115.609290   
                   2017   432.728767  244.750685      123.254795   
                   2018   440.495890  248.487671      126.736986   
        KaggleRama 2015   691.939726  388.676712      196.821918   
                   2016   709.106557  399.095628      201.983607   
                   2017   752.857534  423.035616      214.846575   
                   2018   771.967123  434.317808      219.624658   

product                  Mugs/Sticker  Hats/Sticker  
country store      date                              
Finland KaggleMart 2015      1.985264      3.517490  
                   2016      1.977422      3.527386  
                   2017      1.984117      3.513869  
                   2018      1.980108      3.513222  
        KaggleRama 2015      1.970949      3.499465  
                   2016      1.973011      3.514017  
                   2017      1.975481      3.467783  
                   2018      1.982568      3.480847  
Norway  KaggleMart 2015      1.976348      3.520361  
                   2016      1.964199      3.491771  
                   2017      1.973647      3.510208  
                   2018      1.972467      3.503922  
        KaggleRama 2015      1.974737      3.520385  
                   2016      1.976841      3.499132  
                   2017      1.978461      3.492325  
                   2018      1.976292      3.511321  
Sweden  KaggleMart 2015      1.974560      3.511713  
                   2016      1.967575      3.523267  
                   2017      1.985730      3.510847  
                   2018      1.960656      3.475670  
        KaggleRama 2015      1.974763      3.515562  
                   2016      1.975881      3.510713  
                   2017      1.969013      3.504164  
                   2018      1.977546      3.514938

In [48]:
# Group by month
kk = train_df.groupby(['product', train_df.date.dt.month]).num_sold.mean().unstack(level='product')
kk['Mugs/Sticker'] = kk['Kaggle Mug'] / kk['Kaggle Sticker']
kk['Hats/Sticker'] = kk['Kaggle Hat'] / kk['Kaggle Sticker']
kk

product,Kaggle Hat,Kaggle Mug,Kaggle Sticker,Mugs/Sticker,Hats/Sticker
date,,,,,
1,688.612903,419.631720,187.096774,2.242859,3.680517
2,686.278761,372.073746,169.793510,2.191331,4.041843
3,729.115591,353.951613,172.145161,2.056123,4.235470
4,784.891667,352.075000,184.615278,1.907074,4.251499
5,752.611559,329.176075,186.262097,1.767274,4.040605
6,662.947222,301.909722,178.951389,1.687105,3.704622
7,573.794355,290.754032,172.998656,1.680672,3.316756
8,509.762097,303.047043,171.380376,1.768272,2.974448
9,476.206944,326.465278,172.684722,1.890528,2.757667
